# **FB-CCA**

In [ ]:
from dataset import EEGDataset

data_path = r"path\to\dataset"

# Initialize dataset loader
dataset = EEGDataset(data_path)
print(dir(dataset))  # This should list all available methods
# Load all subjects
dataset.load_all_subjects()
print("Subjects loaded:", dataset.raw_data.keys())

In [ ]:
from preprocessing import EEGPreprocessor

# Initialize preprocessing
sfreq = 1000  # Sampling frequency
preprocessor = EEGPreprocessor(sfreq, l_freq=6.0, h_freq=30.0)

# Define channels of interest
channels_of_interest = ['O1', 'O2', 'Oz', 'Fz']
valid_keys = ['1','2','3','4','5','6','7','8','9']
# Preprocess subjects
epochs_data = {}
for subject, raw in dataset.raw_data.items():
    raw_filtered = preprocessor.filter_data(raw)
    epochs = preprocessor.create_epochs_from_raw(raw_filtered, tmin=0, tmax=4, 
                                                 valid_keys=valid_keys,
                                                 channels_of_interest=channels_of_interest)
    
    epochs_data[subject] = epochs
    print(f"Processed {subject}: {len(epochs)} epochs extracted.")

In [ ]:
from feature_extraction import FeatureExtractor
from classifier import EEGClassifier
from utils import get_event_mapping
import numpy as np

# Feature Extraction (Using CCA with Filterbank)
feature_extractor = FeatureExtractor(method="CCA", filterbank=True, num_harmonics=5)

features = []
labels = []
for subject, epochs in epochs_data.items():
    X = feature_extractor.extract_features(epochs)
    y = epochs.events[:, -1]  # Assuming labels are stored in events
    features.append(X)
    labels.append(y)

# Concatenate features and labels
X_all = np.vstack(features)
y_all = np.concatenate(labels)

# Classification (Leave-Subject-Out using KNN)
classifier = EEGClassifier(n_neighbors=5)
accuracy = classifier.classify(X_all, y_all)

print(f"Classification Accuracy: {accuracy:.4f}")
